In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import csv, random, os, sys, json, requests, re
import numpy as np
import pandas as pd
from openai import OpenAI
from pandas import read_csv
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report

## 1. The onion list

In [ ]:
onion = pd.read_csv("onion_path1")

# concat all the csv files
# onion index 02 ~ 10
for i in range(9):
    onion_index = pd.read_csv(f"onion_path{i+2}.csv")
    onion = pd.concat([onion, onion_index], ignore_index=True)

onion = onion.sort_values(by='Date', ascending=False)
onion = onion.dropna(subset=['Title', 'Content'])
onion = onion[~onion['Content'].str.contains("Previous Slide Next Slide List slides")]
onion = onion[onion['Content'].str.count("Advertisement") < 2]
onion = onion.reset_index(drop=True)

onion = onion.iloc[:1569, :] # for matching the number of data with reddit

In [7]:
onion_list = []
for i in range(len(onion)):
    onion_list.append('[Title]\n' + onion.loc[i].to_list()[1] + '\n' + '[Contents]\n' + onion.loc[i].to_list()[3])

onion_list_len = [len(x) for x in onion_list]
onion_list_len = pd.Series(onion_list_len)
print(onion_list_len.describe())

count    1569.000000
mean      942.959847
std       491.174691
min        66.000000
25%       532.000000
50%      1035.000000
75%      1320.000000
max      2544.000000
dtype: float64


## 2. Reddit list

In [ ]:
reddit = pd.read_csv('reddit_path1')
reddit1 = pd.read_csv('reddit_path2', encoding='utf-8')
reddit2 = pd.read_csv('reddit_path3', encoding='utf-8')
reddit = pd.concat([reddit, reddit1, reddit2])
reddit = reddit.dropna(subset=['Title', 'Content'])
reddit = reddit.reset_index(drop=True)


reddit_list = []
for i in range(len(reddit)):
    reddit_list.append('[Title]\n' + reddit.loc[i].to_list()[1] + '\n' + '[Contents]\n' + reddit.loc[i].to_list()[3])

# statistics
reddit_list_len = [len(x) for x in reddit_list]
reddit_list_len = [len(x) for x in reddit_list]
reddit_list_len = pd.Series(reddit_list_len)
print(reddit_list_len.describe())
print(reddit.columns)

## 3. Make Test

In [10]:
data = []
for i in range(len(onion_list)):
    data.append([onion_list[i], 0])
for i in range(len(reddit_list)):
    data.append([reddit_list[i], 1])


print(f'The length of data: {len(data)}')

random.seed(43)
random.shuffle(data)

The length of data: 3138



## 4. GPT4o

In [ ]:
cot_prompt_txt = """The uploaded text is one of the articles that may be real or fake.
Please Answer whether below article is fake or real. Give a 20-character rationale for why you think that way, and output a 0 and 1 at the end of the sentence.
To Solve this, You have to think step by step. The first step in identifying fake news is evaluating the reliability of the information source. Well-known and verified news organizations are generally more reliable, and their reports can be trusted more than unverified sources.
In addition to source reliability, look at the language used in the content. Fake news often uses sensational or exaggerated language designed to elicit an emotional response. It is also important to check for consistency and accuracy in the information presented; fake news typically includes claims that are either unverified or clearly false. Another critical step is cross-verification,
where check if the same claims are reported by multiple trusted sources.
i.e. rationale + answer 1 if you think the article is real, rationale + answer 0 if you think it is fake.\n Must Keep in mind that the end of a sentence should end with either 0 or 1

###
Output: (rational) + (0||1)
"""

zeroshot_promt_txt = """The uploaded text is one of the articles that may be real or fake.
Please Answer whether below article is fake or real. Say nothing but the number 0 or 1.
i.e. Answer 1 if you think the article is real, answer 0 if you think it is fake.

"""

### 3-2. GPT BatchAPI 

In [88]:
api_key = "your_api_key"
client = OpenAI(api_key=api_key)

In [78]:
# Creating an array of json tasks
tasks_zeroshot = []
tasks_cot = []

# task for zero-shot prompt
for  i in range(len(data)):
    task = {
        "custom_id": f"{i}",
        "method": "POST",
        "url": "/v1/chat/completions",
        "body": {
            "model": "gpt-4o",
            "temperature": 0,
            "messages": [
                {"role": "system", "content": zeroshot_promt_txt},
                {"role": "user", "content": data[i][0]}
            ],
        }
    }
    tasks_zeroshot.append(task)

# task for cot prompt
for i in range(len(data)):
    task = {
        "custom_id": f"{i}",
        "method": "POST",
        "url": "/v1/chat/completions",
        "body": {
            "model": "gpt-4o",
            "temperature": 0,
            "messages": [
                {"role": "system", "content": cot_prompt_txt},
                {"role": "user", "content": data[i][0]}
            ]
        }
    }
    tasks_cot.append(task)


print(f"The original length of tasks: {len(data)}")
print(f"The length of tasks: {len(tasks_zeroshot)}")
print(f"The length of tasks: {len(tasks_cot)}")

The original length of tasks: 3138
The length of tasks: 3138
The length of tasks: 3138


In [79]:
# Creating the file
file_name_zeroshot = "batch_tasks_zeroshot.jsonl"
with open(file_name_zeroshot, 'w') as file:
    for obj in tasks_zeroshot:
        file.write(json.dumps(obj) + '\n')

file_name_cot = "batch_tasks_cot.jsonl"
with open(file_name_cot, 'w') as file:
    for obj in tasks_cot:
        file.write(json.dumps(obj) + '\n')

In [80]:
# Uploading the file
batch_file_zeroshot = client.files.create(
    file=open(file_name_zeroshot, "rb"),
    purpose="batch"
)

batch_file_cot = client.files.create(
    file=open(file_name_cot, "rb"),
    purpose="batch"
)

In [82]:
# Creating the batch job
batch_job_zeroshot = client.batches.create(
  input_file_id=batch_file_zeroshot.id,
  endpoint="/v1/chat/completions",
  completion_window="24h"
)

batch_job_cot = client.batches.create(
  input_file_id=batch_file_cot.id,
  endpoint="/v1/chat/completions",
  completion_window="24h"
)


In [ ]:
# Checking the status of the batch
batch_job_zeroshot = client.batches.retrieve(batch_job_zeroshot.id)
print(batch_job_zeroshot.id)

batch_job_cot = client.batches.retrieve(batch_job_cot.id)

print(batch_job_zeroshot)
print(batch_job_cot)

In [ ]:
# # Retrieving the results
# 1. zeroshot
result_file_zeroshot_id = batch_job_zeroshot.output_file_id
result_zeroshot = client.files.content(result_file_zeroshot_id).content
result_zeroshot_file_name = "data/batch_job_zeroshot_results.jsonl"

with open(result_zeroshot_file_name, 'w') as file:
    file.write(result_zeroshot.decode('utf-8'))

results_zeroshot = []
with open(result_zeroshot_file_name, 'r') as file:
    for line in file:
        json_obj = json.loads(line.strip())
        results_zeroshot.append(json_obj)



# 2. cot
result_file_cot_id = batch_job_cot.output_file_id
print(result_file_cot_id)
result_cot = client.files.content(result_file_cot_id).content
result_cot_file_name = "data/batch_job_cot_results.jsonl"


with open(result_cot_file_name, 'w') as file:
    file.write(result_cot.decode('utf-8'))

results_cot = []
with open(result_cot_file_name, 'r') as file:
    for line in file:
        json_obj = json.loads(line.strip())
        results_cot.append(json_obj)
